In [4]:
(defun dig-pow (n p)
  (loop
    :with m integer = n :with r integer = 0
    :for p integer :downfrom (+ p (floor (log n 10)))
    :do (setf (values m r) (floor m 10))
    :sum (expt r p) :into pow
    :until (zerop m)
    :finally
    (return (multiple-value-bind (k r) (floor pow n)
              (if (zerop r) k -1)))))

DIG-POW

In [5]:
(dig-pow 89 1)

(dig-pow 92 1)

(dig-pow 695 2)

(dig-pow 46288 3)

1

-1

2

51

In [2]:
(expt 5 4)

625